# Tutoriel d'usage de l'API

In [14]:
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [15]:
import numpy as np 

In [6]:
http://51.91.251.0:3000/remaining-requests/12e1e6fc-456b-4738-ba59-b90c3386f583

SyntaxError: invalid syntax (Temp/ipykernel_8168/1042654847.py, line 1)

## Creating an avatar

The admin of the competition can create users. A user can then create as many avatars as they wish.

In [305]:
import requests

user_id = '12e1e6fc-456b-4738-ba59-b90c3386f583'
name = 'ema-swedish2-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [200]>

In [291]:
# name = 'max-second-avatar'
# r = requests.post(path(f"avatars/{user_id}/{name}"))
# r

In [261]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6217 ema-first-avatar
34250 ema-austrian1-avatar
34256 ema-belgian1-avatar
34257 ema-bulgarian1-avatar
34258 ema-croatian1-avatar
34305 ema-german1-avatar
34259 ema-cypriot1-avatar
34260 ema-czech1-avatar
34264 ema-finnish1-avatar
34265 ema-french1-avatar
34261 ema-danish1-avatar
34262 ema-dutch1-avatar
34263 ema-estanian1-avatar
34429 ema-greek1-avatar
34457 ema-irish1-avatar
34455 ema-hungarian2-avatar
34456 ema-hungarian3-avatar
34458 ema-italian1-avatar
35123 ema-belgian3-avatar
37849 ema-luxembourgish1-avatar
38438 ema-luxembourgish2-avatar
73369 ema-maltese1-avatar
73371 ema-polish2-avatar
73370 ema-polish1-avatar
92219 ema-irish2-avatar
92220 ema-italian2-avatar
92221 ema-latvian1-avatar
92222 ema-lithuanian1-avatar
95016 ema-portuguese1-avatar
95247 ema-romanian1-avatar
95248 ema-slovakian1-avatar
95249 ema-slovene1-avatar
95250 ema-spanish1-avatar
95251 ema-swedish1-avatar
96528 ema-austrian2-avatar
96530 ema-cypriot2-avatar
96532 ema-danish2-avatar
96533 ema-dutch2-avatar
965

## Making pricing requests

In [228]:
language_list=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish' ,'swedish']

city_list=['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna','vilnius']

date_list=list(np.arange(1,45,1))

In [229]:
## A modifier logiquement selon dates souhaitées

date_list_soon=[2,5,6,9]

date_list_middle= [15,17,19] 

date_list_away=[27,30,31,33,35,37]



In [306]:
language_test='swedish'
avatar_test='ema-swedish2-avatar'
date_test=date_list_middle

In [307]:
print(avatar_test, language_test,date_test)

ema-swedish2-avatar swedish [15, 17, 19]


In [308]:
requests_list=[]

for date in reversed(date_test) : 
    for city in city_list : 
        params = {
            "avatar_name": avatar_test,
            "language": language_test,
            "city": city,
            "date": date,
            "mobile": np.random.randint(0,2),
        }
        r = requests.get(path(f"pricing/{user_id}"), params=params)
        r.json()
        
        requests_list.append(r)
       

## Build a training set

In [309]:
import pandas as pd

pricing_requests = []

for r in requests_list:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,450,181,54,amsterdam,19,swedish,1,146515
1,891,129,8,amsterdam,19,swedish,1,146515
2,945,162,24,amsterdam,19,swedish,1,146515
3,969,83,13,amsterdam,19,swedish,1,146515
4,917,292,39,amsterdam,19,swedish,1,146515


In [310]:
len(pricing_requests)

2784

In [311]:
# print(pricing_requests[ pricing_requests['mobile'] == 0])

In [312]:
data=pd.read_csv('./Datatrain/test_requete.csv')

In [313]:
data=data.drop(['Unnamed: 0'],axis=1)

In [314]:
pricing_requests=pd.concat([data,pricing_requests])

In [315]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,556,175,60,copenhagen,30,hungarian,0,6217
1,664,284,111,copenhagen,30,hungarian,0,6217
2,96,124,37,copenhagen,30,hungarian,0,6217
3,865,277,107,copenhagen,30,hungarian,0,6217
4,58,203,65,copenhagen,30,hungarian,0,6217
...,...,...,...,...,...,...,...,...
82,620,170,12,vilnius,15,swedish,1,146515
83,605,182,15,vilnius,15,swedish,1,146515
84,516,264,23,vilnius,15,swedish,1,146515
85,490,144,13,vilnius,15,swedish,1,146515


In [316]:
pricing_requests.to_csv('./Datatrain/test_requete.csv')

In [317]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,556,175,60,copenhagen,30,hungarian,0,6217,Boss Western,J.Halliday Inn,1,0,0
1,664,284,111,copenhagen,30,hungarian,0,6217,Accar Hotels,Safitel,0,1,0
2,96,124,37,copenhagen,30,hungarian,0,6217,Boss Western,Boss Western,0,0,0
3,865,277,107,copenhagen,30,hungarian,0,6217,Chillton Worldwide,Chill Garden Inn,1,1,0
4,58,203,65,copenhagen,30,hungarian,0,6217,Yin Yang,Royal Lotus,1,0,2


In [318]:
pricing_requests.to_csv('./Datatrain/test_requete_features.csv')

In [275]:
# sample_submission = pd.read_csv('../input/defi-ia-2023/sample_submission.csv')
# sample_submission.head()

In [118]:
# sample_submission.to_csv('submission.csv', index=False)